# Python爬虫教程

课程来源：[Python超强爬虫8天速成（完整版）爬取各种网站数据实战案例](https://www.bilibili.com/video/BV1ha4y1H7sx?p=1&vd_source=cf36c3cbebd2c2649e023b592f888efe)

## 使用场景

通用爬虫：抓取系统的组成部分，抓取一张页面数据

聚焦爬虫：在通用的基础上，抓取特定内容

增量式爬虫：检测网站中数据更新

## http协议
服务器与客户端进行数据交互的一种形式

常用请求头信息

- User-Agent：请求载体的身份标识

- Connection：请求完毕后，是断开连接还是保持

常用响应头协议
- Content-Type：服务器响应客户端的数据类型

## https协议

- 安全的超文本协议

加密方式

- 对称密钥加密

- 非对称密钥加密

- 证书密钥加密

---

## requests模块

作用：模拟浏览器发送请求

如何使用：

- 指定url

- 发起请求

- 获取响应数据

- 持久化存储

In [1]:
#需求：爬取搜狗首页
import requests
#指定url
url = 'https://www.sogou.com/'
#发起请求
#返回响应对象
response = requests.get(url = url)
#获取响应数据
#获取字符串形式的响应数据
page_text = response.text
print(page_text)
#持久化存储
with open('./sougou.html', 'w', encoding ='utf-8') as fp:
    fp.write(page_text)
    print('爬取结束！')

<!DOCTYPE html><html lang="cn"><head><meta name="viewport" content="width=device-width,minimum-scale=1,maximum-scale=1,user-scalable=no"><script>window._speedMark = new Date();  window.lead_ip = '223.73.20.142';
    window.now = 1656988302196;</script><script type="text/javascript">/*file=static/js/resourceErrorReport.js*/!function(a){var n=(new Date).getTime(),r=a.location.protocol;function c(e,t){var o=(new Date).getTime()-n;(new Image).src=["//pb.sogou.com/pv.gif?uigs_productid=wapapp&type=resource-error&stype=",e,"&timestamp=",o,"&protocol=",r,"&host=",encodeURIComponent(a.location.host),"&path=",encodeURIComponent(a.location.pathname),"&resource=",encodeURIComponent(t)].join("")}function e(e){if((e=e||a.event)&&"error"===e.type){var t=e.srcElement?e.srcElement:e.target;if(t){var o,n,r=t.tagName;"LINK"===r?(n="css",(o=t.getAttribute("href"))&&o.match(/\.css($|\?)/)&&c(n,o)):"SCRIPT"===r&&(n="js",(o=t.getAttribute("src"))&&o.match(/\.js($|\?)/)&&c(n,o))}}}r&&(r=r.substring(0,r.lengt

## 案例

- 指定搜索结果页面

- 破解百度翻译

- 豆瓣电影分类排行榜

- 肯德基餐厅查询

- 国家药监局相关数据

## 指定搜索结果页面

### User-Agent伪装

UA检测：

门户网站的服务器检测对应的请求对应载体身份标识，
若检测到载体的身份标识是某一浏览器，则为正常请求，
若检测载体的身份标识为非浏览器，则异常，可能拒绝请求。

查看方式：

`Shift + Ctrl + I`打开浏览器开发者工具，点击网络，选中网站，点击标头，找到User-Agent

In [2]:
#指定搜索结果页面
import requests
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
url = 'https://www.sogou.com/web'
#处理url携带的参数：封装到字典中
kw = input('enter a word:')
param = {
    'query' : kw
}
#对指定url发起请求是携带参数的，且请求过程中处理了参数
response = requests.get(url = url, params = param, headers = headers)
page_text = response.text
filename = kw + '.html'
with open(filename, 'w', encoding = 'utf-8') as fp:
    fp.write(page_text)
print(filename, '保存成功！')

enter a word:吉吉
吉吉.html 保存成功！


## 破解百度翻译

刷新页面的一部分

### 相关知识

- post请求（携带了参数）

- 响应数据是一组

### 操作

1. `Shift + Ctrl + I`打开开发者工具，点击网络，点击Fetch/XHR

2. 点击名称，点击负载，找到显示关键词的组

3. 返回标头，复制url

4. 在标头中找到Content-Type，显示json则为json方法

In [3]:
#破解百度翻译
import requests
import json
post_url = 'https://fanyi.baidu.com/sug'
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
#post请求参数处理（与get相同）
word = input('enter a word:')
data = {
    'kw' : word
}
#请求发送
response = requests.post(url = post_url, data = data, headers = headers)
#获取相应数据:json方法返回的是object(响应数据为json类型，才能使用)
dic_obj = response.json()
#持久化存储
filename = word + '.json'
fp = open(filename, 'w', encoding = 'utf-8')
json.dump(dic_obj, fp = fp, ensure_ascii = False)
print('over!')

enter a word:武汉
over!


## 豆瓣电影分类排行榜

下拉页面会进行部分刷新，如增加电影数量没滚动条位置变化

In [4]:
#豆瓣电影分类排行榜
import requests
import json
url = 'https://movie.douban.com/j/chart/top_list'
param = {
    'type' : '24',
    'interval_id' : '100:90',
    'action' : '',
    'start' : '0', #从库中第几部电影开始取
    'limit' : '20', #单次请求取出的个数
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
response = requests.get(url = url, params = param, headers = headers)
list_data = response.json()
fp = open('./douban.json', 'w', encoding = 'utf-8')
json.dump(list_data, fp = fp, ensure_ascii = False)
print('over!')

over!


## 作业：肯德基餐厅查询

重点：

1. 确定网址的url

2. 确定post请求还是get请求

3. 确定响应数据是text还是json

In [5]:
#作业：肯德基餐厅查询
import requests
url = 'https://www.kfc.com.cn/kfccda/ashx/GetStoreList.ashx?op=keyword'
word = input('enter a city:')
data = {
    'cname' : '',
    'pid' : '',
    'keyword' : word,
    'pageIndex' : '1',
    'pageSize' : '10',
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
response = requests.post(url = url, data = data, headers = headers)
page_text = response.text
filename = word + '.html'
with open(filename, 'w', encoding = 'utf-8') as fp:
    fp.write(page_text)
print(filename, '保存成功！')

enter a city:广州
广州.html 保存成功！


## 国家药监局基于化妆品生产许可证相关数据

网址：[化妆品生产许可信息管理平台](http://scxk.nmpa.gov.cn:81/xk/)

需求：需要爬取企业的具体信息，非主业列表

并不一定能直接从地址栏中请求得到

可能是**动态加载**出来的数据

对应企业的数据是通过ajax动态请求得到的

**<font color=red>未解决</font>**

In [ ]:
#国家药监局基于化妆品生产许可证相关数据
import requests
url = 'http://scxk.nmpa.gov.cn:81/xk/itownet/portalAction.do?method=getXkzsList'
data = {
    'on': 'true',
    'page': '1',
    'pageSize': '15',
    'productName': '',
    'conditionType': '1',
    'applyname': '',
    'applysn': '',
}
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
id_list = []
json_ids = requests.post(url = url, headers = headers, data = data)
for dic in json_ids['list']:
    id_list.append(dic['ID'])

---

## 数据解析

聚焦爬虫：爬取页面指定内容

分类：正则、bs4、**xpath**

原理：

- 解析局部文本内容在标签之间或标签对应的属性中存储

- 指定标签进行定位

- 标签或标签属性中存储的数据值进行提取（解析）

流程：

1. 指定url

2. 发起请求

3. 获取相应数据

4. 数据解析

5. 持久化存储

In [ ]:
#爬取图片数据
import requests
url = 'http://mari.hzau.edu.cn/__local/B/C9/4C/40EE6E9C872EBEF7B0E864D8540_34AE630B_15EF3.jpg'
#content返回二进制图像数据
#text字符串，content二进制，json对象
img_data = requests.get(url = url).content
with open('./image.jpg', 'wb') as fp:
    fp.write(img_data)

In [ ]:
#爬取所有图片数据
import requests
import re
import os
#创建文件夹
if not os.path.exists('./marilibs'):
    os.mkdir('./marilibs')
#爬取页面信息
url = 'http://mari.hzau.edu.cn/info/1003/3711.htm'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
page_text = requests.post(url = url, headers = headers).text
#获取图像数据
#正则表达式
ex = '<p style="text-align: center"><img src="(.*?)" width'
img_src_list = re.findall(ex, page_text, re.S)
print(img_src_list)
#拼接图片地址
for src in img_src_list:
    src = 'http://mari.hzau.edu.cn' + src
    img_data = requests.get(url = src, headers = headers).content
#存储图片
    img_name = src.split('/')[-1]
    imgPath = './marilibs/' + img_name
    with open(imgPath, 'wb') as fp:
        fp.write(img_data)
        print(img_name, '下载成功！')

## 分页爬取

url当中包含通用，也包含页码参数，通过循环将页码拼接到通用url当中进行循环

网页来源：[【壁纸】抱图吱声 壁纸头像各种类型](https://jump2.bdimg.com/p/4378860308)

In [ ]:
#爬取贴吧壁纸图片数据
import requests
import re
import os
#创建文件夹
if not os.path.exists('./wallpapers'):
    os.mkdir('./wallpapers')
#UA伪装
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
#设置通用url模板
url = 'https://jump2.bdimg.com/p/4378860308?pn=%d'
pageNum = 1
for pageNum in range(1, 5):
    new_url = format(url%pageNum) 
    page_text = requests.post(url = new_url, headers = headers).text
#获取图像数据
#正则表达式
    ex = '<img class="BDE_Image" src="(.*?)" size'
    img_src_list = re.findall(ex, page_text, re.S)
    print(img_src_list)
#拼接图片地址
    for src in img_src_list:
        src = src
        img_data = requests.get(url = src, headers = headers).content
#存储图片
        img_name = src.split('/')[-1]
        imgPath = './wallpapers/' + img_name
        with open(imgPath, 'wb') as fp:
            fp.write(img_data)
            print(img_name, '下载成功！')

## bs4解析

Python独有的解析方式

原理：
- 实例化一个BeautifulSoup对象，并将页面源码加载到该对象中

- 通过调用BeautifulSoup对象中的相关属性或方法对标签进行定位和数据提取

安装：`bs4`，`lxml`

对象的实例化：

- 将本地html文档数据加载到对象当中
    
```python
fp = open('./test.html', 'r', encoding = 'utf-8')
soup = BeautifulSoup(fp, 'lxml')
```

- 从网上获取源码加载到该对象中
    
```python
page_text = response.text    
soup = BeautifulSoup(page_text, 'lxml')
```

提供的用于数据解析的方法和属性：

- `soup.tagName`返回的是文档中第一个出现的tagName标签

- `soup.find`

    - 等同于soup.tagName
    
    - 属性定位：`soup.find('div', class_/id/attr = 'song')`
    
- `soup.find_all(tagName)`返回符合要求的所有标签

- `soup.slelct`

    - `select('某种选择器(id, class, 标签...选择器)')`，返回的是一个列表
    
    - `soup.select('.tang > ul > li > a')[0])`，层级选择器，大于号表示一个层级
    
    -  `print(soup.select('.tang > ul a'))`，空格表示的是多个层级

获取标签之间文本数据：

- `soup.a.text/string/get_text()`

- `text和get_text()`可以获取某一标签下所有的文本内容

- `string`只可以获取该标签下的直系文本

获取标签中属性值：

- `soup.a['href']`

In [ ]:
#将本地html文档数据加载到对象当中
from bs4 import BeautifulSoup
fp = open('./test.html', 'r', encoding = 'utf-8')
soup = BeautifulSoup(fp, 'lxml')
#print(soup)
#print(soup.a) #soup.tagName返回的是html中第一个出现的a标签
#print(soup.div)
#print(soup.find('div')) #等同于soup.div
#print(soup.find('div', class_ = 'song')) #属性定位
#print(soup.find_all('a'))
#print(soup.select('.tang'))
#print(soup.select('.tang > ul > li > a')[0])
#print(soup.select('.tang > ul a')[0].get_text())
#print(soup.select('.tang > ul a')[0]['href'])

In [ ]:
#爬取三国演义小说所有章节的标题和内容
import requests
from bs4 import BeautifulSoup
#爬取首页页面数据
url = 'https://www.shicimingju.com/book/sanguoyanyi.html'
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.30'
}
page_text = requests.get(url = url, headers = headers).content
#解析章节标题与详情页
soup = BeautifulSoup(page_text, 'lxml')
li_list = soup.select('.book-mulu > ul > li')
fp = open('./sanguo.txt', 'w', encoding = 'utf-8')
for li in li_list:
    title = li.a.string
    detail_url = 'https://www.shicimingju.com' + li.a['href']
    #对详情页发起请求
    detail_page_text = requests.get(url = detail_url, headers = headers).content
    #解析详情页内容
    detail_soup = BeautifulSoup(detail_page_text, 'lxml')
    div_tag = detail_soup.find('div', class_ = 'chapter_content')
    content = div_tag.text
    fp.write(title + ':' +content + '\n')
    print(title, '爬取成功！')

## xpath解析

最常用且便捷的解析方式，具备通用性

步骤：

- 实例化etree对象，需要将被解析的页面源码加载到该对象中

- 调用etree对象中的xpath方法结合着xpath、表达式实现标签定位和内容捕获

安装`lxml`

原理：

- 将本地html加载到etree对象中

    etree.parse(filpath)
    
- 将互联网中的源码数据加载

    etree.HTML("page_text')

- xpath('xpath表达式')

    - 从根节点开始定位，用单斜杠表示切换层级
    
    - 双斜杠表示多个层级，可以从任意位置开始定位
    
    - 属性定位：`//tag[@attrName = "attrValue"]`
    
    - 索引定位：`//div[@class = "song"]/p[3]`
    
    - 取文本：
        
        - `text()`获取的是标签中直系的文本内容
    
        - `//text()`标签中非直系的文本内容（所有的文本内容）
    
    - 取属性：`/attrName`相当于`img/src`

In [23]:
from lxml import etree
#实例化一个对象
tree = etree.parse('./test.html')
# r = tree.xpath('/html/body/div') #xpath只能按层级进行定位
# r = tree.xpath('/html//div')
# r = tree.xpath('//div')
# r = tree.xpath('//div[@class = "song"]')
# r = tree.xpath('//div[@class = "song"]/p[3]')
# r = tree.xpath('//div[@class = "tang"]//li[5]/a/text()')[0] #取文本中的第一个元素
# r = tree.xpath('//li[7]//text()')[0]
# r = tree.xpath('//div[@class = "song"]/img/@src')
print(r)

杜牧


In [1]:
#爬取58同城二手房信息
import requests
from lxml import etree
#文本获取
headers = {
    'Uer-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33'
}
url = 'https://www.58.com/ershoufang/'
page_text = requests.get(url = url, headers = headers).text
#数据解析
tree = etree.HTML(page_text)
tr_list = tree.xpath('//div[@class = "cleft"]//tr')
fp = open('58.txt', 'w', encoding = 'utf-8')
for tr in tr_list:
    #局部解析
    title = tr.xpath('./td[2]/a/text()')[0]
    print(title)
    fp.write(title + '\n')

全新装修房源   南北通透   厨卫朝北  
开发商内部清退房源，低于市场价20万，名额有
长宁区 地铁口 精装复式挑高4.5米，民用水
松江菜花泾60弄小区3/6楼学区七中
地铁口的现房，碧桂园业主诚心出售，真实有效欢
民乐城惠益绿苑东苑2室1厅1卫
徐汇商圈万体馆周边宛南二村小户型诚意出售
首付2成，看房有钥匙，班伡接送，送车位，送，
上海五金城复式精装5室3厅2卫1厨
浦东曹路民耀路268弄房屋诚意出售
阳光城小两房，送20万装修家电家具，自带地下
嘉华苑4室3厅3卫自住房源出售
浦东曹路阳光苑民耀路268弄诚意出售
内环内东外滩鑫隆公寓南北全明3室2厅1卫
金地自在城(二期3室2厅1卫
中南品质开发，衡水学/校，沪乍杭高铁旁，物美
199万!满五一!79平环城新村 两室两厅
首付9万 平湖市中区 送车位 高铁旁
日月光水岸花园(公寓3室2厅2卫
首付15万尖山大湾区 长三角发展区域 送车位
静安04年房龄新，投'资，学'区两相宜，房东
顶楼复式 双露台 房型正气 自住装修 诚心出
西郊九韵城2室1厅1卫
首付6万 嘉兴花园电梯洋房 沪乍杭铁路直达 
三面采光卧室客厅均朝南，大露台中心位置实际使
嘉善，首付36万，送车位，百年，配套成熟，
房东急降1048万，装修全送，钥匙房随时看，
景源佳苑，12号线东陆路站，豪华婚房装修70
东盛公寓6复7，三室两厅
本地中介（可落户）外地人可买 送20平花园 


In [15]:
#解析下载图片数据
import requests
from lxml import etree
import os
headers = {
    'Uer-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33'
}
url = 'http://pic.netbian.com/4kmeinv/'
response = requests.get(url = url, headers = headers)
#手动设置数据编码格式
# response.encoding = 'utf-8'
page_text = response.text
#解析src属性值，alt属性值
tree = etree.HTML(page_text)
li_list = tree.xpath('//div[@class = "slist"]/ul/li')
#创建文件夹
if not os.path.exists('./piclibs'):
    os.mkdir('./piclibs')
for li in li_list:
    img_src = 'http://pic.netbian.com' + li.xpath('./a/img/@src')[0]
    img_name = li.xpath('./a/img/@alt')[0] + '.jpg'
    #通用处理中文乱码解决方案
    img_name = img_name.encode('iso-8859-1').decode('gbk')
    print(img_name, img_src)
    #持久化存储
    img_data = requests.get(url = img_src, headers = headers).content
    img_path = './piclibs/' + img_name
    with open(img_path, 'wb') as fp:
        fp.write(img_data)
        print(img_name, '下载成功！')

[<Element li at 0x1edb9846540>, <Element li at 0x1edb9846100>, <Element li at 0x1edb9846040>, <Element li at 0x1edb9846140>, <Element li at 0x1edb98460c0>, <Element li at 0x1edb9846200>, <Element li at 0x1edb9846240>, <Element li at 0x1edb98462c0>, <Element li at 0x1edb9846300>, <Element li at 0x1edb98461c0>, <Element li at 0x1edb9846340>, <Element li at 0x1edb9846380>, <Element li at 0x1edb9846280>, <Element li at 0x1edb98463c0>, <Element li at 0x1edb9845180>, <Element li at 0x1edb9845140>, <Element li at 0x1edb9845100>, <Element li at 0x1edb9845080>, <Element li at 0x1edb9845040>, <Element li at 0x1edb98456c0>]
居家 浴缸 浴巾 美女 牛仔裤 4k壁纸.jpg http://pic.netbian.com/uploads/allimg/220707/233455-1657208095aec4.jpg
居家 浴缸 浴巾 美女 牛仔裤 4k壁纸.jpg 下载成功！
古韵 气质 美女 美腿 古装 森系4k美女壁纸.jpg http://pic.netbian.com/uploads/allimg/220715/153854-16578707348791.jpg
古韵 气质 美女 美腿 古装 森系4k美女壁纸.jpg 下载成功！
cosplay美女 电脑桌 键盘 机房 可爱 小姐姐4k壁纸3840x2160.jpg http://pic.netbian.com/uploads/allimg/210831/102129-16303764895142.jpg
cosp

In [3]:
#爬取全国城市的名称
import requests
from lxml import etree
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33'
}
url = 'https://www.aqistudy.cn/historydata/'
page_text = requests.get(url = url, headers = headers).text
tree = etree.HTML(page_text)
all_city_names = []
#解析热门城市
hot_city_list = tree.xpath('//div[@class = "bottom"]/ul/li')
for li in hot_city_list:
    hot_city_name = li.xpath('./a/text()')[0]
    all_city_names.append(hot_city_name)
#解析所有城市
city_names_list = tree.xpath('//div[@class = "bottom"]/ul/div[2]/li')
for li in city_names_list:
    city_name = li.xpath('./a/text()')[0]
    all_city_names.append(city_name)
print(all_city_names, len(all_city_names))

['北京', '上海', '广州', '深圳', '杭州', '天津', '成都', '南京', '西安', '武汉', '阿坝州', '安康', '阿克苏地区', '阿里地区', '阿拉善盟', '阿勒泰地区', '安庆', '安顺', '鞍山', '克孜勒苏州', '安阳', '蚌埠', '白城', '保定', '北海', '宝鸡', '北京', '毕节', '博州', '白山', '百色', '保山', '白沙', '包头', '保亭', '本溪', '巴彦淖尔', '白银', '巴中', '滨州', '亳州', '长春', '昌都', '常德', '成都', '承德', '赤峰', '昌吉州', '五家渠', '昌江', '澄迈', '重庆', '长沙', '常熟', '楚雄州', '朝阳', '沧州', '长治', '常州', '潮州', '郴州', '池州', '崇左', '滁州', '定安', '丹东', '东方', '东莞', '德宏州', '大理州', '大连', '大庆', '大同', '定西', '大兴安岭地区', '德阳', '东营', '黔南州', '达州', '德州', '儋州', '鄂尔多斯', '恩施州', '鄂州', '防城港', '佛山', '抚顺', '阜新', '阜阳', '富阳', '抚州', '福州', '广安', '贵港', '桂林', '果洛州', '甘南州', '固原', '广元', '贵阳', '甘孜州', '赣州', '广州', '淮安', '海北州', '鹤壁', '淮北', '河池', '海东地区', '邯郸', '哈尔滨', '合肥', '鹤岗', '黄冈', '黑河', '红河州', '怀化', '呼和浩特', '海口', '呼伦贝尔', '葫芦岛', '哈密地区', '海门', '海南州', '淮南', '黄南州', '衡水', '黄山', '黄石', '和田地区', '海西州', '河源', '衡阳', '汉中', '杭州', '菏泽', '贺州', '湖州', '惠州', '吉安', '金昌', '晋城', '景德镇', '金华', '西双版纳州', '九江', '吉林', '即墨', '江门', '荆门', '佳木斯', '济南', '济宁', '胶南', '酒泉', '句容', '湘西州', '

In [4]:
#爬取全国城市的名称
import requests
from lxml import etree
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33'
}
url = 'https://www.aqistudy.cn/historydata/'
page_text = requests.get(url=url, headers=headers).text
tree = etree.HTML(page_text)
#解析到热门城市和所有城市对应的a标签
# //div[@class="bottom"]/ul/li/          热门城市a标签的层级关系
# //div[@class="bottom"]/ul/div[2]/li/a  全部城市a标签的层级关系
a_list = tree.xpath('//div[@class="bottom"]/ul/li/a | //div[@class="bottom"]/ul/div[2]/li/a')
all_city_names = []
for a in a_list:
    city_name = a.xpath('./text()')[0]
    all_city_names.append(city_name)
print(all_city_names,len(all_city_names))

['北京', '上海', '广州', '深圳', '杭州', '天津', '成都', '南京', '西安', '武汉', '阿坝州', '安康', '阿克苏地区', '阿里地区', '阿拉善盟', '阿勒泰地区', '安庆', '安顺', '鞍山', '克孜勒苏州', '安阳', '蚌埠', '白城', '保定', '北海', '宝鸡', '北京', '毕节', '博州', '白山', '百色', '保山', '白沙', '包头', '保亭', '本溪', '巴彦淖尔', '白银', '巴中', '滨州', '亳州', '长春', '昌都', '常德', '成都', '承德', '赤峰', '昌吉州', '五家渠', '昌江', '澄迈', '重庆', '长沙', '常熟', '楚雄州', '朝阳', '沧州', '长治', '常州', '潮州', '郴州', '池州', '崇左', '滁州', '定安', '丹东', '东方', '东莞', '德宏州', '大理州', '大连', '大庆', '大同', '定西', '大兴安岭地区', '德阳', '东营', '黔南州', '达州', '德州', '儋州', '鄂尔多斯', '恩施州', '鄂州', '防城港', '佛山', '抚顺', '阜新', '阜阳', '富阳', '抚州', '福州', '广安', '贵港', '桂林', '果洛州', '甘南州', '固原', '广元', '贵阳', '甘孜州', '赣州', '广州', '淮安', '海北州', '鹤壁', '淮北', '河池', '海东地区', '邯郸', '哈尔滨', '合肥', '鹤岗', '黄冈', '黑河', '红河州', '怀化', '呼和浩特', '海口', '呼伦贝尔', '葫芦岛', '哈密地区', '海门', '海南州', '淮南', '黄南州', '衡水', '黄山', '黄石', '和田地区', '海西州', '河源', '衡阳', '汉中', '杭州', '菏泽', '贺州', '湖州', '惠州', '吉安', '金昌', '晋城', '景德镇', '金华', '西双版纳州', '九江', '吉林', '即墨', '江门', '荆门', '佳木斯', '济南', '济宁', '胶南', '酒泉', '句容', '湘西州', '

## 作业：站长素材中免费简历模板



In [44]:
import requests
from lxml import etree
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.63 Safari/537.36 Edg/102.0.1245.33'
}
url = 'https://sc.chinaz.com/jianli/free.html'
page_text = requests.get(url = url, headers = headers).text
tree = etree.HTML(page_text)
url_list = tree.xpath('//div[@id = "container"]//p')
alt_list = tree.xpath('//div[@id = "container"]/div')
if not os.path.exists('./cv'):
    os.mkdir('./cv')
for url in url_list:
    cv_url = 'https:' + url.xpath('.//a/@href')[0]
    cv_text = requests.get(url = cv_url, headers = headers).text
    tree = etree.HTML(cv_text)
    download_url = tree.xpath('//div[@id = "down"]/div[2]/ul/li[3]/a/@href')[0]
    cv_name = tree.xpath('//div[@class = "bggray clearfix"]//h1')
    for name in cv_name:
        download_name = name.xpath('./text()')[0]
        download_name = download_name.encode('iso-8859-1').decode('utf-8')
        cv_path = 'cv/' + download_name + '.rar'
        cv_data = requests.get(url = download_url,headers = headers).content
        with open(cv_path, 'wb') as fp:
            fp.write(cv_data)
            print(download_name, '下载成功')

高级财务经理简历模板免费下载 下载成功
水彩背景个人简历表格模板word 下载成功
工程造价自考个人简历表格 下载成功
小升初免费简历模板电子版 下载成功
办公室文员岗位求职简历模板 下载成功
房地产买卖经纪人简历模板下载 下载成功
韩语专业个人简历模板范文 下载成功
机电技术与应用个人简历模板 下载成功
软件技术电子版个人简历表格 下载成功
销售经理简历模板免费下载 下载成功
会计出纳个人简历表格模板 下载成功
在校学生个人简历封面模板 下载成功
应届生药剂师大学生简历模板 下载成功
语文老师个人简历模板范文 下载成功
个人简历封面模板简洁 下载成功
微商自我介绍简短模板 下载成功
个人简历封面图片可编辑 下载成功
学前教育个人简历表格样式 下载成功
应届建筑工程简历制作免费模板 下载成功
物业管理个人简历模板范文 下载成功
